# How the Embedding Layers in BERT Were Implemented (BERT의 임베딩 레이어가 구현 된 방법) 





### Introduction
이번장에서는 버트의 임베딩 레이어(Token Embedding, Segment Embedding, Positional Embedding)등을 설명해 보도록 하겠습니다


### Overview

다음은 BERT에서 각 포함레이어의 기능을 적절하게 설명한 다이어그램입니다.

!['title'](https://miro.medium.com/max/1548/1*iJqlhZz-g6ZQJ53-rE9VvA.png)

NPL 관련 작업 해결을 위한 대부분의 딥러닝 모델과 마찬가지로 BERT는 각 input token (입력 텍스트의 단어)를 Token Embedding Layer를 통해 전달하여 각 토큰이 Vector표현으로 변환되게 됩니다.  

하지만 다른 딥러닝 모델과는 다르게, BERT 는 추가적인 Embedding Layer(Segment Embedding, Positional Embedding)들을 가지고 있습니다. 이러한 추가 Embdding Layer들은 논문의 끝에서는 명확해 질 것입니다.  

## Token Embedding

#### purpose

이전 섹션(?? 뭐징?)에서 언급했던것 처럼, Token Embeddings Layer의 역할은 단어를 고정된 차원의 벡터 표현으로 변환해 주는것입니다.  
Bert의 경우, 각 단어는 768 차원의 벡터로 표현됩니다.

#### Implementaion

"I like strawberries" 라는 문장이 있다고 가정하자. 아래 그림은 Token Embedding Layer를 설명 하고 있따

!['token_embedding'](https://miro.medium.com/max/2070/1*BXjLEnX89ftmFO_l91sP-A.png)

(질문 : straw와 berries 를 보면 subword 단위로 나눈거 같은데 뭘 쓴거지?) 아래 정답이 나옴  


이 Input text 는 Token Embedding Layer에 들어가기 전에 Token화가 됩니다.  
또한 시작 ([CLS]) 및 끝 ([SEP])와 같은 추가적인 토큰이 Token화 된 문장에 추가가 됩니다.  
위에 토큰화는 WordPiece Tokenization을 통해 수행됩니다.  
WordPiece Tokenization는 다음에 알아보도록 하겠습니다. :3   

이러한 Token Embedding Layer는 각 wordpiece token을 768차원의 벡터표현으로 변환합니다. 그결과 우리의 6개의 input token들은 (6,768)의 matrix 으로 전환되고, 만약 batch axis 를 포함한다면 (1,6,768)크기의 텐서로 변환되게 됩니다,   
batch axis : ??  

## Segment Embeddings (부분 임베딩?)

#### Purpose
BERT 는 한쌍의 입력 텍스트가 주어졌을때 텍스트 분류와 관련된 NLP작업을 가능케 합니다.  
예를들어 두개의 텍스트가 의미적으로 유사한 지 여부를 분류하는 문제를 들 수 있을것 입니다.  
한쌍의 입력 텍스트들은 단순하게 연결되어 MODEL에 전달되게 됩니다.  
그렇다면 BERT는 어떻게 주어진 쌍의 입력을 구별할 수 있을까요??  
정답은 Segment Embedding 입니다.  


!['segment_embedding](https://miro.medium.com/max/1593/1*7p1kuvoafkCziIWYZSG94w.png)

이 Segment Embedding Layer는 오직 2개의 벡터 표현만 가지고 있습니다.  
첫 벡터 (index 0) 에는 입력1에 속하는 모든 토큰이 할당되고 마지막 벡터는 입력2에 속하는 모든 토큰에 할당됩니다.  

만약 input값이 단순히 하나의 문장으로만 구성되어있다면, 그러면 이 input 에 대한 sement embdding 은 Segment Embdding Table 의 0에 해당하는 벡터일 뿐입니다.
?? 이게 뭔소리지,,

## Positional Embedding 

#### Purpose 
BERT 는 트랜스포머 스택으로 구성되어있으며, Transformer는 입력의 순차적인 특성을 인코딩 하지 않습니다.  
예를 들어 아래와 같은 문장이 있다고 합시다.  
I think, therefore I am

첫번쨰 "I" 는 절대로 두번째 "I" 와 동일한 벡터 표현을 가져서는 안됩니다.


#### Implementaion 
BERT는 최대 512길이의 Input Sequence를 처리하도록 설계 되어 있습니다.  
The authors incorporated the sequential nature of the input sequences by having BERT learn a vector representation for each position.  
이 말은 Position Embedding Layer의 크기가 (512,768)의 lookup table 이며, 첫 번째 행은 첫 번째 위치에 있는 단어의 벡터 표현이고, 두 번째 행은 두 번째 위치에 있는 단어 벡터의 표현이라는 것입니다.  
그러므로, "Hello world" 및 "Hi there"과 같은 입력이 있는 경우 "Hello" 및 "Hi"는 입력 시퀀스의 첫 단어이므로 동일한 Postitional Embedding 값을 갖는다고 합니다.  
동이랗게 "world" 와 "there" 또한 같은 postion embedding 값을 갖습니다.

## Combining Representations  

우리는 지금까지 길이가 n인 토큰화 된 입력 sequence가 다음과 같은 세 가지 고유한 표현을 갖는 것을 보았습니다.   
- Token Embeddings with shape (1, n, 768) which are just vector representations of words
- Segment Embeddings with shape (1, n, 768) which are vector representations to help BERT distinguish between paired input sequences.
- Position Embeddings with shape (1, n, 768) to let BERT know that the inputs its being fed with have a temporal property.
        temporal property : 시간적인 특성
        
이러한 값들은 요소별로 합쳐져 (1, n, 768)크기의 단일값을 생성합니다.  
이것들은 BERT의 ENCODER LAYER에 전달되는 입력 값 잆니다.


원글주소 : https://medium.com/@_init_/why-bert-has-3-embedding-layers-and-their-implementation-details-9c261108e28a